<a href="https://colab.research.google.com/github/SilviaDani/treeCSPsolver/blob/main/AIproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Point:
  def __init__(self, x, y, name):
    self.x = x
    self.y = y
    self.name = name


In [ ]:
import numpy as np
class SegmentMatrix:
  def __init__(self, n):
    self.matrix = np.zeros((n, n), dtype = int)
    for i in range(n):
      self.matrix[i, i] = -1

  def insert_segment(self, p1_name, p2_name):
    self.matrix[p1_name, p2_name] = 1
    self.matrix[p2_name, p1_name] = 1

  def shape(self):
    return self.matrix.shape

In [ ]:
import random
import math 
import matplotlib.pyplot as plt
from collections import defaultdict

class Map:
  def reset(self):
    self.Y = []  
    self.segments_paramenters = defaultdict(list)

  def isThere_point(self, x, y):
    for point in self.Y:
      if x == point.x and y == point.y:
        return True
    return False

  def generate_points(self, n):
    for i in range(n):
      x = random.random()
      y = random.random()
      if self.isThere_point(x, y):
        self.generate_points(1)
      else:
        new_point = Point(x, y, i)
        self.Y.append(new_point)

  def is_intersected(self, p3, p4):
    epsilon = 0.00000000000001
    px_min = min(p3.x, p4.x)
    px_max = max(p3.x, p4.x)
    py_min = min(p3.y, p4.y)
    py_max = max(p3.y, p4.y)
    A1 = 1
    den1 = p3.x - p4.x
    if den1 != 0:
      m1 = (p3.y - p4.y) / den1 
      q1 = p3.y - m1 * p3.x
    else:
      A1 = 0
      m1 = 1
      q1 = -p3.x
    for i in range(len(self.segments_paramenters)):
      for segment in self.segments_paramenters[i]:
          if A1 == segment[4] and m1 == segment[5] and q1 == segment[6]:
            if (px_min < segment[7].x and segment[7].x < px_max and py_min < self.segment[7].y and segment[7].y < py_max) or (px_min < segment[8].x and segment[8].x < px_max and py_min < segment[8].y and segment[8].y < py_max):
              return True
          elif A1 != segment[4] or m1 != segment[5]:
            a_list = [[-m1, A1], [- segment[5], segment[4]]]
            A = np.array(a_list)
            B = np.array ([q1, segment[6]])
            intersection = np.linalg.solve(A, B)
            x = intersection[0]
            y = intersection[1]
            if px_min + epsilon < x < px_max - epsilon and py_min + epsilon < y < py_max - epsilon and segment[0] + epsilon < x < segment[1] - epsilon and segment[2] + epsilon < y < segment[3] - epsilon:
              return True
    return False

  def find_connection(self, p, matrix):
    min_distance = 3
    min_i = -1
    for i in range(matrix.shape[0]):
      if matrix[p.name, i] == 0:
        distance = pow(p.x - self.Y[i].x, 2) + pow(p.y - self.Y[i].y, 2)
        if not self.is_intersected(p, self.Y[i]):
          if distance < min_distance:
            min_distance = distance
            min_i = i
        else:
          matrix[p.name, i] = -1
    return min_i

  def are_connections_possible(self, matrix):
    for i in range(matrix.shape[0]):
      for j in range(matrix.shape[0]):
        if matrix[i, j] == 0:
          return True
    return False

  def generate_map(self, n):
    self.reset()
    matrix = SegmentMatrix(n)
    self.generate_points(n)
    i = 0
    while self.are_connections_possible(matrix.matrix):
      chosen_i = np.random.randint(0, (len(self.Y)))
      chosen_point = self.Y[chosen_i]
      while self.find_connection(chosen_point, matrix.matrix) == -1 and self.are_connections_possible(matrix.matrix):
        connection = self.find_connection(chosen_point, matrix.matrix)
        chosen_i = np.random.randint(0, (len(self.Y)))
        chosen_point = self.Y[chosen_i]
      if self.find_connection(chosen_point, matrix.matrix) != -1:
        connection = self.find_connection(chosen_point, matrix.matrix)
        matrix.matrix[chosen_i, connection] = 1
        matrix.matrix[connection, chosen_i] = 1
        segmentx_min = min(chosen_point.x, self.Y[connection].x)
        segmentx_max = max(chosen_point.x, self.Y[connection].x)
        segmenty_min = min(chosen_point.y, self.Y[connection].y)
        segmenty_max = max(chosen_point.y, self.Y[connection].y)
        A2 = 1
        den2 = chosen_point.x - self.Y[connection].x
        if den2 != 0:
          m2 = (chosen_point.y - self.Y[connection].y) / den2
          q2 = chosen_point.y - m2 * chosen_point.x
        else:
          A2 = 0
          m2 = 1
          q2 = - chosen_point.x
        self.segments_paramenters[i].append([segmentx_min, segmentx_max, segmenty_min, segmenty_max, A2, m2, q2, chosen_point, connection])
    self.matrix = matrix


In [ ]:
class Tree:
  def __init__(self, map):
    self.vertexNum = len(map.Y)
    self.MST = defaultdict(list)
    self.edges = np.zeros((self.vertexNum, self.vertexNum), dtype = int)
    for row in range(self.vertexNum):
      for column in range(row + 1):
        if map.matrix.matrix[row, column] == 1:
          weight = random.randint(1, 999)
          self.edges[row, column] = weight
          self.edges[column, row] = weight
        else:
          self.edges[row, column] = 0
          self.edges[column, row] = 0

  def primsAlgorithm(self, map):
    visited = [0] * self.vertexNum
    edgeNum = 0
    index_root = random.randint(0, self.vertexNum - 1)
    visited[index_root] = 1
    while edgeNum < self.vertexNum - 1:
      min = 9999999
      for i in range(self.vertexNum):
        if visited[i] == 1:
          for j in range(self.vertexNum):
            if visited[j] == 0 and self.edges[i, j] != 0:
              if min > self.edges[i, j]:
                min = self.edges[i, j]
                s = i
                d = j
        else:
          continue
      self.MST[s].append(d)
      self.MST[d].append(s)
      visited[d] = 1
      edgeNum += 1

In [ ]:
class Variable:
  def __init__(self, tree):
    self.neighbors = defaultdict(list)
    for node in range(len(tree.MST)):
      for neighbor in tree.MST[node]:
        self.neighbors[node].append(neighbor)
    self.parents = [None] * len(tree.MST)
    self.children = defaultdict(list)

In [ ]:
class Constraint:
  def __init__(self, MST):
    self.constraints = []
    for i in range(len(MST)):
      for j in MST[i]:
        self.constraints.append((i, j))

  def respected_constraint(self, di, dj):
    if di != dj:
      return True
    return False

In [ ]:
class CSP:
  def __init__(self, tree):
    self.X = Variable(tree)
    self.n = len(tree.MST)
    self.D = defaultdict(list)
    for node in range(len(tree.MST)):
      for color in range(2):
        self.D[node].append(color)
    self.C = Constraint(tree.MST)
    self.assignment = [None] * self.n
 
  def topologicalSortUtil(self,v,visited,stack):
    visited[v] = 1
    for i in self.X.neighbors[v]:
      if visited[i] == 0:
        self.X.parents[i] = v
        self.X.children[v].append(i)
        self.topologicalSortUtil(i, visited, stack)
    stack.insert(0, v)

  def topologicalSort(self, root):
    visited = [0] * self.n
    stack =[]
    self.topologicalSortUtil(root, visited, stack)
    return stack
 
  def check_domain(self, di, i, j):
    respected = False
    remove_fromDomain = []
    for dj in self.D[j]:
      if self.C.respected_constraint(di, dj):
        respected = True
      elif len(self.D[i]) == 1:
        remove_fromDomain.append(dj)
    return [respected, remove_fromDomain]

  def revise(self, i, j):
    revised = False
    respected = True
    for di in self.D[i]:
      domain = self.check_domain(di, i, j)
      if not domain[0]:
        respected = False
      elif len(domain[1]) != 0:
        revised = True
        for color in domain[1]:
          self.D[j].remove(color)
    return [respected, revised]

  def make_arc_consistent(self, parent, child):
    revision = self.revise(parent, child)
    if not revision[0]:
      return False
    elif revision[1]:
      if self.X.children[child]:
        for node in self.X.children[child]:
          self.make_arc_consistent(child, node)
    return True
  
  def tree_CSP_solver(self):
    root = random.randint(0, self.n - 1)
    sorted_variables = self.topologicalSort(root)
    for current_vertex in reversed(sorted_variables):
      if self.X.parents[current_vertex] != None:
        if not self.make_arc_consistent(self.X.parents[current_vertex], current_vertex):
          return None
    for node in sorted_variables:
      consistent_value = self.D[node]
      if consistent_value:
        self.assignment[node] = consistent_value[0]
        if len(consistent_value) > 0:
          self.D[node].clear()
          self.D[node].append(self.assignment[node])
        if self.X.children[node]:
          for child in self.X.children[node]:
            if not self.make_arc_consistent(node, child):
              return None
      else:
        return None
    return self.assignment


In [ ]:
import time
random.seed(3)
np.random.seed(3)
n_number = np.arange(10, 260, 50)
avg_times = []
max_times = []
new_map = Map()
n_rep = 7
for n in n_number:
  print(f'n={n}')
  total_rep_time = []
  for i in range(n_rep):
    new_map.generate_map(n)
    new_tree = Tree(new_map)
    new_tree.primsAlgorithm(new_map)
    new_CSP = CSP(new_tree)
    start = time.perf_counter()
    new_CSP.tree_CSP_solver()
    stop = time.perf_counter()
    computation_time = stop - start
    total_rep_time.append(computation_time)
  avg_times.append(np.mean(total_rep_time))
plt.plot(n_number, avg_times)
plt.ylabel('time')
plt.xlabel('n')
plt.savefig('output.jpg')
plt.show()